In [2]:
import pandas as pd
import holidays

# 1. 데이터 불러오기
file_path = '/Users/jung0202/Desktop/인천공항/airport_daily_flight_merged.csv'
df = pd.read_csv(file_path)

# 2. 날짜 형식 변환 (YYYY.M.D 형식 -> datetime 객체)
# 오류 방지를 위해 errors='coerce'를 사용하여 변환 불가능한 값은 NaT로 처리 후 제거
df['date_obj'] = pd.to_datetime(df['date'], format='%Y.%m.%d', errors='coerce')
df = df.dropna(subset=['date_obj']) # 날짜가 없는 행 제거

# 3. 한국 공휴일 정보 가져오기 (1990년부터 2030년까지 미리 로드)
# years 인자를 넉넉하게 주어 데이터 범위 내 모든 연도의 휴일을 계산합니다.
min_year = df['date_obj'].dt.year.min()
max_year = df['date_obj'].dt.year.max()
kr_holidays = holidays.KR(years=range(min_year, max_year + 1))

def get_day_type(row):
    """
    날짜를 입력받아 평일(0), 휴일(1), 설(2), 추석(3)으로 분류하는 함수
    우선순위: 설/추석 > 일반 공휴일/주말 > 평일
    """
    dt = row['date_obj']
    
    # 공휴일 여부 및 이름 확인
    is_holiday = dt in kr_holidays
    holiday_name = kr_holidays.get(dt) if is_holiday else ""
    
    # 1순위: 설날 (대체공휴일 포함)
    if is_holiday and '설날' in holiday_name:
        return 2
    
    # 2순위: 추석 (대체공휴일 포함)
    if is_holiday and '추석' in holiday_name:
        return 3
    
    # 3순위: 일반 휴일 (주말 또는 설/추석을 제외한 공휴일)
    # 주말: 토요일(5), 일요일(6)
    if is_holiday or dt.weekday() >= 5:
        return 1
    
    # 4순위: 평일
    return 0

# 4. 함수 적용하여 새로운 칼럼 생성
df['day_type'] = df.apply(get_day_type, axis=1)

# 5. 불필요한 임시 칼럼 삭제 및 결과 저장
df = df.drop(columns=['date_obj'])
output_filename = '/Users/jung0202/Desktop/인천공항/airport_daily_flight_with_daytype.csv'
df.to_csv(output_filename, index=False)

# 결과 확인
print(f"파일이 저장되었습니다: {output_filename}")
print(df[['date', 'day_type']].head(10))
print("\n[day_type 분포]")
print(df['day_type'].value_counts().sort_index())

파일이 저장되었습니다: /Users/jung0202/Desktop/인천공항/airport_daily_flight_with_daytype.csv
        date  day_type
0   2002.1.1         1
1   2002.1.2         0
2   2002.1.3         0
3   2002.1.4         0
4   2002.1.5         1
5   2002.1.6         1
6   2002.1.7         0
7   2002.1.8         0
8   2002.1.9         0
9  2002.1.10         0

[day_type 분포]
day_type
0    5921
1    2629
2      77
3      78
Name: count, dtype: int64
